In [13]:
from sqlconnector import SQLConnector
import os
import pandas as pd

machineJsonFile = 'machines.json'
accessoriesJsonFile = 'machine-accessories.json'

firebaseImportPrg = 'C:\\Users\\loser.ralph\\workspace\\mhm-firebase-import'
databaseFilePath = 'database-files'
importPrg = 'json-to-firestore.js'

In [14]:

sqlConnector = SQLConnector('copyManager', '|~qpc,wm*of/=%"$i7h!i; m2~i4vab^')
conn = sqlConnector.create_connection()

## Artikel Abfrage
articleQuery = 'select AATR.[No_] as \'itemNo\',ART.[Description] as \'descrGER\',ENU.[Description] as \'descrENG\',ART.[Base Unit of Measure] as \'quantity\',BATR.[Name] as \'machineFamily\',CATR.[Value] as \'sort\', ART.[Unit Price] as \'listPrice\' from [MHM$Item Attribute Value Mapping] as AATR join [MHM$Item Attribute] as BATR on AATR.[Item Attribute ID] = BATR.[ID] join [MHM$Item Attribute Value] as CATR on CATR.[Attribute ID] = AATR.[Item Attribute ID] join [MHM$Item] as ART on AATR.[No_] = ART.[No_] join [MHM$Item Translation] as ENU on AATR.[No_] = ENU.[Item No_] where (AATR.[Item Attribute ID] < 39) and (ENU.[Language Code] = \'ENU\') and (AATR.[Item Attribute Value ID] = CATR.[ID]) order by AATR.[No_] asc'

df = pd.read_sql(articleQuery, conn)
print(df.head())
machine_accessory = df[df['machineFamily'].str.contains("Maschine Zubehör")]
machine_accessory.set_index('itemNo', inplace=True)

          itemNo                             descrGER  \
0  20-0-0-0017-0        Rakelaufnahmeschiene komplett   
1  20-0-0-0096-0  Siebaufnahme U-Profil Y-Achse 50x70   
2  20-0-0-0096-0  Siebaufnahme U-Profil Y-Achse 50x70   
3  20-0-0-0096-0  Siebaufnahme U-Profil Y-Achse 50x70   
4  20-0-0-0096-0  Siebaufnahme U-Profil Y-Achse 50x70   

                      descrENG quantity                   machineFamily sort  \
0  squeegee pivot bar complete       ST            Maschine SP5000 AUTO    1   
1  U-profile adapter bar 50x70       ST         Maschine SP5000 Manuell    1   
2  U-profile adapter bar 50x70       ST            Maschine SP5000 AUTO    1   
3  U-profile adapter bar 50x70       ST  Maschine SP5000 Manuell VAKUUM    1   
4  U-profile adapter bar 50x70       ST     Maschine SP5000 AUTO VAKUUM    1   

   listPrice  
0       84.0  
1       73.0  
2       73.0  
3       73.0  
4       73.0  


In [15]:
#negated
#Machine accessories with machine family as array
machines = df[~df['machineFamily'].str.contains("Maschine Zubehör")]

grouped_machine_family = machines.groupby(['itemNo']).machineFamily.apply(';'.join).to_frame()

accessories = df[df['machineFamily'].str.contains("Maschine Zubehör")]

merged = pd.merge(accessories, grouped_machine_family, on=['itemNo'])
merged.drop('machineFamily_x', axis=1, inplace=True)
merged.rename(columns={'machineFamily_y': 'machineFamily'}, inplace=True)
merged['machineFamily'] = merged['machineFamily'].str.replace('Maschine ', '')
merged['listPrice'] = merged['listPrice'].round(2)

In [16]:
import json
res = json.loads(merged.to_json(orient='records'))
with open(firebaseImportPrg + os.sep + databaseFilePath + os.sep + accessoriesJsonFile, 'w') as outfile:
    json.dump(res, outfile)

In [17]:
####################################################################################
########################### M a c h i n e s ########################################
####################################################################################

In [18]:
machines = df[df['itemNo'].str.startswith('S')]
machines = machines.rename(columns={'sort': 'tableNo'})

machines['machineFamily'] = machines['machineFamily'].str.replace('Maschine ', '')

machineFamilies = machines[~machines['machineFamily'].str.match('(Anzahl Tische)|[0-9]{2,3}x[0-9]{2,3}')][['itemNo', 'machineFamily']]
machineFamilies = machineFamilies.set_index('itemNo')

In [19]:
tableNumbers = machines[machines['machineFamily'].str.contains('Anzahl Tische')][['itemNo', 'tableNo']]
tableNumbers = tableNumbers.set_index('itemNo')

In [20]:
printSizes = df[df['itemNo'].str.startswith('S')]
printSizes['machineFamily'] = printSizes['machineFamily'].str.replace('Maschine ', '')
printSizes.rename(columns={'machineFamily': 'printSize'}, inplace=True)
printSizes = printSizes[printSizes['printSize'].str.match('[0-9]{2,3}x[0-9]{2,3}')][['itemNo', 'printSize']]
printSizes.set_index('itemNo', inplace=True)

C:\Users\LOSER~1.RAL\AppData\Local\Temp/ipykernel_52200/4171649076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  printSizes['machineFamily'] = printSizes['machineFamily'].str.replace('Maschine ', '')
c:\Python310\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [21]:
machines.drop('machineFamily', inplace=True, axis=1)
machines2 = pd.merge(machines, machineFamilies,  on=['itemNo'])
machines2 = pd.merge(machines2, printSizes, on=['itemNo'])

In [22]:
machines2.drop('tableNo', inplace=True, axis=1)
machines3 = pd.merge(machines2, tableNumbers, on=['itemNo'])


In [23]:
machines3 = machines3.drop_duplicates(subset=['itemNo'])
res = json.loads(machines3.to_json(orient='records'))
print(os.path)
with open(firebaseImportPrg + os.sep + databaseFilePath + os.sep + machineJsonFile, 'w') as outfile:
    json.dump(res, outfile)

machines3.head()

<module 'ntpath' from 'c:\\Python310\\lib\\ntpath.py'>


,itemNo,descrGER,descrENG,quantity,listPrice,machineFamily,printSize,tableNo
0,SI4-50X70,Single Print Station 50x70cm [gesperrt],single print station 50x70 cm,ST,11467.0,Single Print WAND,50x70,1
3,SI4-50X70DP,Single Print Station double pallet 50x70cm,single print station double pallet 50x70cm,ST,11100.0,Single Print WAND,50x70,2
6,SI4-50X70DPV,Single Print Station double pallet 50x70cm Vakuum,single print station double pallet 50x70cm vacuum,ST,15100.0,Single Print WAND VAKUUM,50x70,2
9,SI4-50X70W,Single Print Station fahrbar 50x70cm,single print station moveable 50x70,ST,8100.0,Single Print FAHRBAR,50x70,1
12,SI4-50X70WV,Single Print Station fahrbar 50x70 Vakuum,single print station moveable vacuum 50x70,ST,9300.0,Single Print FAHRBAR VAKUUM,50x70,1


In [24]:
import subprocess
import_process = subprocess.Popen(['node', firebaseImportPrg + os.sep + importPrg], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout,stderr = import_process.communicate()

stderr

KeyboardInterrupt: 